# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [73]:
import pandas as pd 
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [74]:
customer.head(2)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47


In [75]:
uselog_months.head(2)

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2


In [76]:
year_months = list(uselog_months['年月'].unique())
uselog = pd.DataFrame()
for i in range(1,len(year_months)):
    tmp = uselog_months.loc[uselog_months['年月']==year_months[i]]
    tmp.rename(columns={'count':'count_0'},inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月']==year_months[i-1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count':'count_1'},inplace=True)
    tmp = pd.merge(tmp,tmp_before,on='customer_id',how='left')
    uselog = pd.concat([uselog,tmp],ignore_index=True)
    

In [77]:
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [78]:
#退会を未然に防ぐ事が目的なので退会申請する確率を予測する
#退会した顧客に絞り込んでend_date列の一ヶ月前の年月を取得しuselogと結合する
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])


In [79]:
exit_customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,NaT
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,NaT
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,NaT
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,NaT
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,OA143589,XXXXX,C03,F,2019-01-15,2019-02-28,CA1,1,ナイト,6000,通常,4.5,4.5,6,3,0,2019-02-28,1,NaT
4050,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,オールタイム,10500,通常,7.0,6.0,12,3,0,2019-03-31,2,NaT
4068,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,デイタイム,7500,通常,7.0,7.0,11,3,0,2019-03-31,1,NaT
4086,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,ナイト,6000,通常,7.0,7.0,10,4,0,2019-03-31,1,NaT


In [80]:
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
len(uselog)
exit_customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,年月
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30,201803
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30,201804
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30,201804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,OA143589,XXXXX,C03,F,2019-01-15,2019-02-28,CA1,1,ナイト,6000,通常,4.5,4.5,6,3,0,2019-02-28,1,2019-01-28,201901
4050,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,オールタイム,10500,通常,7.0,6.0,12,3,0,2019-03-31,2,2019-02-28,201902
4068,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,デイタイム,7500,通常,7.0,7.0,11,3,0,2019-03-31,1,2019-02-28,201902
4086,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,ナイト,6000,通常,7.0,7.0,10,4,0,2019-03-31,1,2019-02-28,201902


In [81]:
exit_uselog.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [82]:
exit_uselog = exit_uselog.dropna(subset=["name"])
exit_uselog

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30758,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,6000.0,通常,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28
30787,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6000.0,入会費無料,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28
30827,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,7500.0,入会費無料,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28
30842,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,7500.0,通常,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28


In [83]:
len(exit_uselog)

1104

In [84]:
len(exit_uselog['customer_id'].unique())

1104

### ノック43：継続顧客のデータを作成しよう

In [85]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog,conti_customer,on=['customer_id'],how='left')
len(conti_uselog)

33851

In [86]:
conti_uselog = conti_uselog.dropna(subset=['name'])
len(conti_uselog)

27422

In [87]:
#退会データは1104件、継続顧客のデータは数ヶ月にわたって重複している。データをシャッフルして重複を削除する
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
conti_uselog.head()

#https://note.nkmk.me/python-pandas-sample/
#https://note.nkmk.me/python-pandas-reset-index/

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,TS007439,6,8.0,XXX,C03,M,2017-03-01,NaN,CA1,...,ナイト,6000.0,通常,6.166667,6.0,8.0,5.0,1.0,2019-04-30,25.0
1,201806,GD866704,5,6.0,XX,C03,F,2017-04-01,NaN,CA1,...,ナイト,6000.0,通常,4.750000,5.0,7.0,1.0,1.0,2019-04-30,24.0
2,201809,AS058623,6,3.0,XXXXX,C01,F,2015-05-01,NaN,CA1,...,オールタイム,10500.0,通常,4.500000,5.0,6.0,3.0,1.0,2019-04-30,47.0
3,201811,HI570349,4,5.0,XXXXX,C01,F,2015-05-01,NaN,CA1,...,オールタイム,10500.0,通常,4.916667,5.0,7.0,2.0,1.0,2019-04-30,47.0
4,201811,GD381952,5,6.0,XXX,C02,F,2017-08-01,NaN,CA2,...,デイタイム,7500.0,入会費半額,5.916667,6.0,8.0,4.0,1.0,2019-04-30,20.0


In [88]:
len(conti_uselog)

2842

In [89]:
#継続顧客、退会顧客のデータを縦に結合 
predict_data = pd.concat([conti_uselog,exit_uselog],ignore_index=True)
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201810,TS007439,6,8.0,XXX,C03,M,2017-03-01,NaN,CA1,...,6000.0,通常,6.166667,6.0,8.0,5.0,1.0,2019-04-30,25.0,NaT
1,201806,GD866704,5,6.0,XX,C03,F,2017-04-01,NaN,CA1,...,6000.0,通常,4.750000,5.0,7.0,1.0,1.0,2019-04-30,24.0,NaT
2,201809,AS058623,6,3.0,XXXXX,C01,F,2015-05-01,NaN,CA1,...,10500.0,通常,4.500000,5.0,6.0,3.0,1.0,2019-04-30,47.0,NaT
3,201811,HI570349,4,5.0,XXXXX,C01,F,2015-05-01,NaN,CA1,...,10500.0,通常,4.916667,5.0,7.0,2.0,1.0,2019-04-30,47.0,NaT
4,201811,GD381952,5,6.0,XXX,C02,F,2017-08-01,NaN,CA2,...,7500.0,入会費半額,5.916667,6.0,8.0,4.0,1.0,2019-04-30,20.0,NaT


<img src="2021-02-21 6.15.01.png">


In [90]:
len(predict_data)

3946

### ノック44：予測する月の在籍期間を作成しよう

In [91]:
#在籍期間の列(period)を追加する now_dateに（年月）を入れて差を計算
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['年月'],format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i],predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years*12 + delta.months)
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201810,TS007439,6,8.0,XXX,C03,M,2017-03-01,NaN,CA1,...,6.166667,6.0,8.0,5.0,1.0,2019-04-30,25.0,NaT,19,2018-10-01
1,201806,GD866704,5,6.0,XX,C03,F,2017-04-01,NaN,CA1,...,4.750000,5.0,7.0,1.0,1.0,2019-04-30,24.0,NaT,14,2018-06-01
2,201809,AS058623,6,3.0,XXXXX,C01,F,2015-05-01,NaN,CA1,...,4.500000,5.0,6.0,3.0,1.0,2019-04-30,47.0,NaT,40,2018-09-01
3,201811,HI570349,4,5.0,XXXXX,C01,F,2015-05-01,NaN,CA1,...,4.916667,5.0,7.0,2.0,1.0,2019-04-30,47.0,NaT,42,2018-11-01
4,201811,GD381952,5,6.0,XXX,C02,F,2017-08-01,NaN,CA2,...,5.916667,6.0,8.0,4.0,1.0,2019-04-30,20.0,NaT,15,2018-11-01


### ノック45：欠損値を除去しよう

In [92]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               270
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [93]:
#end_date,exit_dateは退会していない人は欠損値になるcount_1の欠損値を除外する
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2624
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2624
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

<img src="2021-02-21 7.06.34.png">

In [94]:
target_col = ['campaign_name' , 'class_name' , 'gender','count_1','routine_flg','period','is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,ナイト,M,8.0,1.0,19,0.0
1,通常,ナイト,F,6.0,1.0,14,0.0
2,通常,オールタイム,F,3.0,1.0,40,0.0
3,通常,オールタイム,F,5.0,1.0,42,0.0
4,入会費半額,デイタイム,F,6.0,1.0,15,0.0


In [95]:
#ダミー変数の作成
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,8.0,1.0,19,0.0,0,0,1,0,0,1,0,1
1,6.0,1.0,14,0.0,0,0,1,0,0,1,1,0
2,3.0,1.0,40,0.0,0,0,1,1,0,0,1,0
3,5.0,1.0,42,0.0,0,0,1,1,0,0,1,0
4,6.0,1.0,15,0.0,1,0,0,0,1,0,1,0


 <img src='スクリーンショット 2021-02-21 7.19.46.png'>


In [96]:
del predict_data['campaign_name_通常']
del predict_data['class_name_ナイト']
del predict_data['gender_M']
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,8.0,1.0,19,0.0,0,0,0,0,0
1,6.0,1.0,14,0.0,0,0,0,0,1
2,3.0,1.0,40,0.0,0,0,1,0,1
3,5.0,1.0,42,0.0,0,0,1,0,1
4,6.0,1.0,15,0.0,1,0,0,1,1


### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [97]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted']==1]
#contiの数はexitの数に合う様に調整
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exit))

X = pd.concat([exit,conti],ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train,y_train)
y_test_pred = model.predict(X_test)
y_test_pred

array([0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 0., 1., 1.

In [98]:
results_test = pd.DataFrame({'y_test':y_test ,"y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
1701,0.0,0.0
7,1.0,1.0
1175,0.0,0.0
793,1.0,1.0
1570,0.0,0.0


### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

In [99]:
#model.scoreを使用しない計算方法
correct = len(results_test.loc[results_test['y_test']==results_test['y_pred']])
data_count = len(results_test)
score_test = correct/data_count
score_test

0.9125475285171103

In [100]:
model.score(X_test,y_test)

0.9125475285171103

In [101]:
model.score(X_train,y_train)

0.9816223067173637

 <img src='スクリーンショット 2021-02-23 6.54.36.png'>

In [102]:
X = pd.concat([exit,conti],ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y)
#max_depthを5とし決定技の深さを５階層までにする
model = DecisionTreeClassifier(random_state=0,max_depth=5)
model.fit(X_train,y_train)
model.score(X_train,y_train)

0.9315589353612167

In [103]:
model.score(X_test,y_test)

0.9239543726235742

### ノック49：モデルに寄与している変数を確認しよう

In [104]:
#model.feature_importances_で重要変数を取得する
importance = pd.DataFrame({'feature_names':X.columns,"coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.368113
1,routine_flg,0.116387
2,period,0.515226
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000274
6,class_name_デイタイム,0.000000
7,gender_F,0.000000


### ノック50：顧客の退会を予測しよう

In [108]:
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [109]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [110]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]


array([[0., 1.]])